# Goals Option 1

## How much are nearby Hospitals and ASCs paid for the same procedure?
Filters for Physician/Other Supplier
Entity Type = O
Provider Type = Ambulatory Surgical Center
CPT = 43249
Filters for Hospital Outpatient
APC = 5302
Compare results within a CBSA

# Stretch Goals

## Extend to other HCPCS/APCs
- Best place to start is to find an APC that is well-populated in the Hospital file
- Next find a well-populated HCPCS from the APC in the Physician/Other Supplier File

## Compare Doctor Utilization for a procedure
- Filters for Physician/Other Supplier
  - Entity Type = I
  - Place of Service = O
- HCPCS = 99213 (or any office visit – 99211-99215 / 99201-99205)
- How many different patients does each specialty typically see in a year? How does that vary by CBSA?

# To Do List

- Create full_name column for providers and outpatients
- I'm getting zero rows when I add the hcpcs filter to providers

In [3]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
# set up dataframes
providers_tsv = '../data/Medicare_Provider_Util_Payment_PUF_CY2017.tsv'
outpatient_csv = '../data/MUP_OHP_R19_P04_V10_D17_APC_Provider.csv'
classification1_csv = '../data/508-Compliant-Version-of-2020_january_web_addendum_b.12312019.csv'
classification2_csv = '../data/2020_january_web_addendum_b.12312019.csv'
cbsa_csv = '../data/ZIP_CBSA_032020.csv'

# NOTE: providers_csv is a TAB-DELIMITED file, Use sep='\t'
providers = pd.read_csv(providers_tsv, sep='\t', low_memory=False) 
outpatient = pd.read_csv(outpatient_csv, low_memory=False) 
classification1 = pd.read_csv(classification1_csv) 
classification2 = pd.read_csv(classification2_csv) 
cbsa = pd.read_csv(cbsa_csv) 

# NOTE: the first line in providers_tsv is copyright info
# providers.head()

In [5]:
# Inner join on HCPS code will eliminate any duplicates
# NOTE: Trailing spaces were found in column names, they're included below and will be removed later
# memory usage after join only increased by 0.5 MB

classifications = pd.merge(left=classification1, right=classification2, 
                           how='inner', 
                           on=['HCPCS Code','Short Descriptor','SI','APC ',
                               'Relative Weight','Payment Rate ','National Unadjusted Copayment ',
                              'Minimum Unadjusted Copayment '])
# classifications.info()

In [6]:
# column names contain newline characters, remove them
# also convert spaces to underscores and make lowercase
outpatient.columns = outpatient.columns.str.replace('\n', '_').str.replace(' ', '_').str.lower()
providers.columns = providers.columns.str.replace(' ', '_').str.lower()
classifications.columns = classifications.columns.str.replace(' ', '_').str.lower()
cbsa.columns = cbsa.columns.str.replace(' ', '_').str.lower()

providers.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,average_medicare_standard_amt
0,1,CPT copyright 2016 American Medical Associatio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99217,Hospital observation care discharge,N,100.0,96.0,100.0,73.398800,325.780000,56.827200,57.492400
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99218,Hospital observation care typically 30 minutes,N,26.0,25.0,26.0,100.080000,449.000000,78.460000,79.306154
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99219,Hospital observation care typically 50 minutes,N,52.0,51.0,52.0,136.380000,614.000000,102.807692,103.895385
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99220,Hospital observation care typically 70 minutes...,N,59.0,59.0,59.0,190.363729,755.932203,141.293559,142.865763


In [7]:
outpatient.head()

,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zip_code,provider_hospital_referral_region_(hrr),apc,apc_description,beneficiaries,comprehensive_apc_services,average_estimated_total_submitted_charges,average_medicare_allowed_amount,average_medicare_payment_amount,outlier_comprehensive_apc_services,average_medicare_outlier_amount
0,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5072,Level 2 Excision/ Biopsy/ Incision and Drainage,249,259,"$9,575","$1,038",$826,NaN,NaN
1,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5073,Level 3 Excision/ Biopsy/ Incision and Drainage,52,53,"$12,578","$1,793","$1,423",NaN,NaN
2,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5091,Level 1 Breast/Lymphatic Surgery and Related P...,26,27,"$11,338","$2,114","$1,684",0,$0
3,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5092,Level 2 Breast/Lymphatic Surgery and Related P...,23,23,"$17,116","$3,737","$2,978",0,$0
4,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,36301,AL - Dothan,5112,Level 2 Musculoskeletal Procedures,17,17,"$7,383","$1,029",$820,0,$0


# Create Full_Name column

## This needs to be added before the individual dataframes are set up
- IF Entity_Type = O THEN Last_Name
- IF Entity_Type = I THEN First_Name + “ “ + Middle Initial + “ “ + Last_Name + “, “ + Credentials


In [71]:
# create full_name column
# df['Price'] = [1500 if x =='Music' else 800 for x in df['Event']] 

outpatient['full_name'] = [last_name if x == 'O'
                          else first_name + ' ' + middle_initial + ' ' + last_name
                           for x in outpatient['entity_type']
                          ]
outpatient.head()

KeyError: 'entity_type'

# Set up City Dataframes

We're examining 3 cities: Nashville, Los Angeles, New York City. Starting with Nashville, we set up dataframes and take a look at the procedures done there.

In [28]:
# set up Nashville Outpatient Dataframe
tennessee_outpatient = outpatient[outpatient ['provider_state'] == 'TN'] 
nashville_outpatient = tennessee_outpatient[tennessee_outpatient ['provider_city'] == 'Nashville'] 
nashville_outpatient

,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zip_code,provider_hospital_referral_region_(hrr),apc,apc_description,beneficiaries,comprehensive_apc_services,average_estimated_total_submitted_charges,average_medicare_allowed_amount,average_medicare_payment_amount,outlier_comprehensive_apc_services,average_medicare_outlier_amount
49924,440006,Tristar Skyline Medical Center,3441 Dickerson Pike,Nashville,TN,37207,TN - Nashville,5072,Level 2 Excision/ Biopsy/ Incision and Drainage,77,82,"$16,659","$1,135",$898,0,$0
49925,440006,Tristar Skyline Medical Center,3441 Dickerson Pike,Nashville,TN,37207,TN - Nashville,5073,Level 3 Excision/ Biopsy/ Incision and Drainage,26,26,"$25,117","$2,010","$1,602",0,$0
49926,440006,Tristar Skyline Medical Center,3441 Dickerson Pike,Nashville,TN,37207,TN - Nashville,5091,Level 1 Breast/Lymphatic Surgery and Related P...,19,20,"$28,844","$2,245","$1,770",0,$0
49927,440006,Tristar Skyline Medical Center,3441 Dickerson Pike,Nashville,TN,37207,TN - Nashville,5092,Level 2 Breast/Lymphatic Surgery and Related P...,11,12,"$39,405","$4,134","$3,294",0,$0
49928,440006,Tristar Skyline Medical Center,3441 Dickerson Pike,Nashville,TN,37207,TN - Nashville,5112,Level 2 Musculoskeletal Procedures,18,19,"$14,519","$1,061",$836,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51293,440197,Tristar Southern Hills Medical Center,391 Wallace Rd,Nashville,TN,37211,TN - Nashville,5431,Level 1 Nerve Procedures,17,19,"$24,275","$1,401","$1,104",0,$0
51294,440197,Tristar Southern Hills Medical Center,391 Wallace Rd,Nashville,TN,37211,TN - Nashville,8011,Comprehensive Observation Services,118,121,"$18,325","$1,983","$1,559",0,$0
51296,440218,Saint Thomas Hospital For Specialty Surgery,2011 Murphy Avenue,Nashville,TN,37203,TN - Nashville,5114,Level 4 Musculoskeletal Procedures,325,334,"$22,373","$4,805","$3,812",0,$0
51297,440218,Saint Thomas Hospital For Specialty Surgery,2011 Murphy Avenue,Nashville,TN,37203,TN - Nashville,5115,Level 5 Musculoskeletal Procedures,13,13,"$50,129","$6,004","$4,688",0,$0


In [30]:
# set up Nashville Providers Dataframe
tennesee_providers = providers[providers ['nppes_provider_state'] == 'TN'] 
nashville_providers = tennesee_providers[tennesee_providers ['nppes_provider_city'] == 'NASHVILLE'] 
nashville_providers

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,average_medicare_standard_amt
2396,1003013947,OTTO,ROBERT,J,M.D.,M,I,2400 PATTERSON STREET SUITE 100,NaN,NASHVILLE,...,20610,Aspiration and/or injection of large joint or ...,N,61.0,37.0,60.0,64.691967,268.032787,47.191311,54.860164
2397,1003013947,OTTO,ROBERT,J,M.D.,M,I,2400 PATTERSON STREET SUITE 100,NaN,NASHVILLE,...,27447,Repair of knee joint,N,26.0,23.0,26.0,1307.530385,4319.000000,1021.338846,1142.162692
2398,1003013947,OTTO,ROBERT,J,M.D.,M,I,2400 PATTERSON STREET SUITE 100,NaN,NASHVILLE,...,72170,"X-ray of pelvis, 1 or 2 views",N,65.0,65.0,65.0,29.410000,78.000000,21.150000,25.368308
2399,1003013947,OTTO,ROBERT,J,M.D.,M,I,2400 PATTERSON STREET SUITE 100,NaN,NASHVILLE,...,73502,"X-ray of hip with pelvis, 2-3 views",N,65.0,39.0,65.0,38.310000,114.000000,28.781692,31.684308
2400,1003013947,OTTO,ROBERT,J,M.D.,M,I,2400 PATTERSON STREET SUITE 100,NaN,NASHVILLE,...,73562,"X-ray of knee, 3 views",N,43.0,36.0,43.0,35.302326,107.441860,25.268372,28.054884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9845871,1992985949,BECK,ROBERT,L,M.D,M,I,2400 PATTERSON ST,SUITE 400,NASHVILLE,...,G0008,Administration of influenza virus vaccine,N,52.0,52.0,52.0,23.730000,67.000000,23.260000,25.320000
9845872,1992985949,BECK,ROBERT,L,M.D,M,I,2400 PATTERSON ST,SUITE 400,NASHVILLE,...,G0439,"Annual wellness visit, includes a personalized...",N,91.0,91.0,91.0,109.980000,312.000000,107.780000,115.360000
9846017,1992987085,HORTON,SUSANNE,M,CRNA,F,I,1211 MEDICAL CENTER DR,NaN,NASHVILLE,...,00320,Anesthesia for procedure on esophagus and neck...,N,17.0,16.0,17.0,186.568824,2194.941176,146.270588,153.111765
9846018,1992987085,HORTON,SUSANNE,M,CRNA,F,I,1211 MEDICAL CENTER DR,NaN,NASHVILLE,...,00740,Anesthesia for procedure on gastrointestinal t...,N,61.0,58.0,60.0,76.923443,917.975410,60.307377,63.126557


In [38]:
# Set up New York City and Los Angeles dataframes
ny_outpatient = outpatient[outpatient ['provider_state'] == 'NY'] 
nyc_outpatient = ny_outpatient[ny_outpatient ['provider_city'] == 'New York'] 
nyc_outpatient

ca_outpatient = outpatient[outpatient ['provider_state'] == 'CA'] 
la_outpatient = ca_outpatient[ca_outpatient ['provider_city'] == 'Los Angeles'] 
la_outpatient

ny_providers = providers[providers ['nppes_provider_state'] == 'NY'] 
nyc_providers = ny_providers[ny_providers ['nppes_provider_city'] == 'NEW YORK'] 
nyc_providers

ca_providers = providers[providers ['nppes_provider_state'] == 'CA'] 
la_providers = ca_providers[ca_providers ['nppes_provider_city'] == 'LOS ANGELES'] 
la_providers

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,average_medicare_standard_amt
125,1003000639,BENHARASH,PEYMAN,NaN,MD,M,I,UCLA CARDIOTHORACIC SURGERY 10833 LECONTE AVE,62-182 CHS,LOS ANGELES,...,93314,Interpretation and report of heart ultrasound ...,N,14.0,14.0,14.0,98.87,452.500000,77.520000,69.220000
1118,1003007576,MOGHISSI,ETIE,NaN,MD,F,I,12555 W JEFFERSON BLVD STE 301,NaN,LOS ANGELES,...,82962,Blood glucose (sugar) test performed by hand-h...,N,429.0,180.0,429.0,3.21,24.477855,3.109394,3.142657
1119,1003007576,MOGHISSI,ETIE,NaN,MD,F,I,12555 W JEFFERSON BLVD STE 301,NaN,LOS ANGELES,...,83036,Hemoglobin A1C level,N,373.0,174.0,373.0,13.32,39.887399,12.848740,13.015013
1120,1003007576,MOGHISSI,ETIE,NaN,MD,F,I,12555 W JEFFERSON BLVD STE 301,NaN,LOS ANGELES,...,95250,Ambulatory continuous glucose (sugar) monitori...,N,111.0,89.0,111.0,180.58,400.000000,154.148018,121.592523
1121,1003007576,MOGHISSI,ETIE,NaN,MD,F,I,12555 W JEFFERSON BLVD STE 301,NaN,LOS ANGELES,...,95251,Ambulatory continuous glucose (sugar) includin...,N,132.0,92.0,132.0,46.21,100.000000,39.786061,34.691667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9845992,1992986947,ANAKWENZE,VICKI,M,"M.D,",F,I,5260 S FIGUEROA ST,216,LOS ANGELES,...,93000,Routine EKG using at least 12 leads including ...,N,15.0,13.0,15.0,18.19,85.000000,13.038667,12.600000
9845993,1992986947,ANAKWENZE,VICKI,M,"M.D,",F,I,5260 S FIGUEROA ST,216,LOS ANGELES,...,99214,Established patient office or other outpatient...,N,130.0,35.0,130.0,113.34,150.000000,81.567692,79.875769
9845994,1992986947,ANAKWENZE,VICKI,M,"M.D,",F,I,5260 S FIGUEROA ST,216,LOS ANGELES,...,99215,Established patient office or other outpatient...,N,17.0,17.0,17.0,152.05,200.000000,113.681176,111.697647
9847378,1992999031,NAIK,RAHUL,R,M.D.,M,I,2020 ZONAL AVE,IRD ROOM 620,LOS ANGELES,...,96375,Injection of different drug or substance into ...,N,30.0,11.0,23.0,24.58,54.333333,19.270000,17.723667


# Filter dataframes

In [65]:
## Filter outpatient
# APC = 5302

# individual statements for filters
nashville_outpatient = nashville_outpatient.loc[
    nashville_outpatient['apc'] == 5302
]

nyc_outpatient = nyc_outpatient.loc[
    nyc_outpatient['apc'] == 5302
]

la_outpatient = la_outpatient.loc[
    la_outpatient['apc'] == 5302
]

# nyc_outpatient.head()

In [70]:
## Filter providers
# SELECT entity type = 'O'
# Provider Type = Ambulatory Surgical Center
# CPT/HCPCS = 43249
# DEBUGDEBUG: I'm getting zero rows when I add the hcpcs filter <-------------------------------------


# individual statements for filters
nashville_providers = nashville_providers.loc[
    (nashville_providers['nppes_entity_code'] == 'O') &
    (nashville_providers['provider_type'] == 'Ambulatory Surgical Center')
#    & (nashville_providers['hcpcs_code'] == 43249)
]

nyc_providers = nyc_providers.loc[
    (nyc_providers['nppes_entity_code'] == 'O') &
    (nyc_providers['provider_type'] == 'Ambulatory Surgical Center')
#     (nyc_providers['hcpcs_code'] == 43249)
]

la_providers = la_providers.loc[
    (la_providers['nppes_entity_code'] == 'O') &
    (la_providers['provider_type'] == 'Ambulatory Surgical Center')
#    & (la_providers['hcpcs_code'] == 43249)
]

nashville_providers.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,average_medicare_standard_amt
738359,1073598066,UROLOGY SURGERY CENTER LP,NaN,NaN,NaN,NaN,O,2801 CHARLOTTE AVE,NaN,NASHVILLE,...,50590,Shock wave crushing of kidney stones,N,113.0,99.0,113.0,1679.970000,4232.000000,1262.982655,1372.380354
738360,1073598066,UROLOGY SURGERY CENTER LP,NaN,NaN,NaN,NaN,O,2801 CHARLOTTE AVE,NaN,NASHVILLE,...,52000,Diagnostic examination of the bladder and blad...,N,939.0,864.0,939.0,271.358019,602.188498,206.589010,217.061416
738361,1073598066,UROLOGY SURGERY CENTER LP,NaN,NaN,NaN,NaN,O,2801 CHARLOTTE AVE,NaN,NASHVILLE,...,52005,Insertion of catheter into urinary duct (urete...,N,18.0,18.0,18.0,616.153333,2042.000000,454.638333,509.323333
738362,1073598066,UROLOGY SURGERY CENTER LP,NaN,NaN,NaN,NaN,O,2801 CHARLOTTE AVE,NaN,NASHVILLE,...,52204,Biopsy of the bladder using an endoscope,N,61.0,52.0,61.0,695.915410,2042.000000,538.895246,565.114918
738363,1073598066,UROLOGY SURGERY CENTER LP,NaN,NaN,NaN,NaN,O,2801 CHARLOTTE AVE,NaN,NASHVILLE,...,52234,Destruction and/or removal of (0.5 to 2.0 cent...,N,24.0,23.0,24.0,1092.442500,2042.000000,827.009583,897.372500


# These Dataframes are Loaded and Ready for Use

## Outpatients
- nashville_outpatient
- nyc_outpatient
- la_outpatient

## Providers
- nashville_providers
- nyc_providers
- la_providers

In [68]:
nashville_outpatient

,provider_id,provider_name,provider_street_address,provider_city,provider_state,provider_zip_code,provider_hospital_referral_region_(hrr),apc,apc_description,beneficiaries,comprehensive_apc_services,average_estimated_total_submitted_charges,average_medicare_allowed_amount,average_medicare_payment_amount,outlier_comprehensive_apc_services,average_medicare_outlier_amount
49935,440006,Tristar Skyline Medical Center,3441 Dickerson Pike,Nashville,TN,37207,TN - Nashville,5302,Level 2 Upper GI Procedures,13,15,"$14,899","$1,249",$995,0,$0
50269,440039,Vanderbilt University Medical Center,1211 Medical Center Drive,Nashville,TN,37232,TN - Nashville,5302,Level 2 Upper GI Procedures,477,623,"$7,121","$1,206",$959,NaN,NaN
50660,440082,Saint Thomas West Hospital,"4220 Harding Rd, Po Box 380",Nashville,TN,37205,TN - Nashville,5302,Level 2 Upper GI Procedures,189,207,"$6,350","$1,225",$971,0,$0
50790,440111,Metro Nashville General Hospital,1818 Albion Street,Nashville,TN,37208,TN - Nashville,5302,Level 2 Upper GI Procedures,11,12,"$4,489","$1,249",$995,0,$0
50923,440133,Saint Thomas Midtown Hospital,2000 Church St,Nashville,TN,37236,TN - Nashville,5302,Level 2 Upper GI Procedures,76,98,"$8,420","$1,238",$985,NaN,NaN
51056,440161,Tristar Centennial Medical Center,2300 Patterson Street,Nashville,TN,37203,TN - Nashville,5302,Level 2 Upper GI Procedures,151,170,"$11,862","$1,208",$954,0,$0
51288,440197,Tristar Southern Hills Medical Center,391 Wallace Rd,Nashville,TN,37211,TN - Nashville,5302,Level 2 Upper GI Procedures,44,49,"$20,274","$1,228",$974,0,$0
